# Dev contrastive knowledge assesment notebook for LLaMa


## Dependencies

In [7]:
!git clone https://github.com/daniel-furman/Capstone.git

Cloning into 'Capstone'...
remote: Enumerating objects: 311, done.
remote: Counting objects: 100% (311/311), done.
remote: Compressing objects: 100% (190/190), done.
remote: Total 311 (delta 145), reused 247 (delta 82), pack-reused 0
Receiving objects: 100% (311/311), 21.14 MiB | 17.10 MiB/s, done.
Resolving deltas: 100% (145/145), done.


In [8]:
!pip install -r /content/Capstone/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 104.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.3/76.3 MB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 103.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 25.5 MB/s eta 0:00:00


## Imports

In [9]:
import os
import torch

In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## LLaMa specific setup

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
!git clone https://github.com/daniel-furman/transformers_llama.git

Cloning into 'transformers_llama'...
remote: Enumerating objects: 124157, done.
remote: Total 124157 (delta 0), reused 0 (delta 0), pack-reused 124157
Receiving objects: 100% (124157/124157), 123.09 MiB | 14.85 MiB/s, done.
Resolving deltas: 100% (93895/93895), done.


In [14]:
# only need to run when using a model for first time

# !python /content/transformers_llama/src/transformers/models/llama/convert_llama_weights_to_hf.py \
    # --input_dir '/content/drive/MyDrive/Colab Files/llama/LLaMA/' \
    # --model_size 7B \
    # --output_dir '/content/drive/MyDrive/Colab Files/llama/LLaMA/int8'

## Run the LLaMas 🦙🦙

In [13]:
os.chdir('/content/transformers_llama')
!pip install .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/transformers_llama
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.27.0.dev0-py3-none-any.whl size=6688316 sha256=1f6878035c5ff85dc0a59b55299cc7180ccf23f2c2310f9303035f9e7ec81c11
  Stored in directory: /tmp/pip-ephem-wheel-cache-i8vc5o52/wheels/86/c1/27/9d840f9c03869c3b830926e03c647575ae0f506156db708a64
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.26.1
    Uninstalling transformers-4.26.1:
      Successfully uninstalled transformers-4.26.1


In [15]:
import transformers as transformers_LLaMa

In [16]:
tokenizer = transformers_LLaMa.LLaMATokenizer.from_pretrained(
    "/content/drive/MyDrive/Colab Files/llama/LLaMA/int8/tokenizer/"
)

model = transformers_LLaMa.LLaMAForCausalLM.from_pretrained(
    "/content/drive/MyDrive/Colab Files/llama/LLaMA/int8/llama-7b/",
    load_in_8bit=True,
    device_map="auto"
)


Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

In [17]:
batch = tokenizer(
    "Steve Jobs is the founder of",
    return_tensors="pt",
    add_special_tokens=False
)

batch = {k: v.cuda() for k, v in batch.items()}
generated = model.generate(batch["input_ids"], max_length=10)
print(tokenizer.decode(generated[0]))

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1201: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


Steve Jobs is the founder of Apple Inc.
